In [2]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://127.0.0.1:5000"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.tracking_uri

'http://127.0.0.1:5000'

In [7]:
from mlflow.entities import ViewType

client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

[<Run: data=<RunData: metrics={'rmse': 2.5783958849462065}, params={'learning_rate': '0.07821770025674853',
  'max_depth': '50',
  'min_child_weight': '10.308529401219346',
  'reg_alpha': '0.3657738575412868',
  'reg_lambda': '0.2780900571916324'}, tags={'mlflow.runName': 'blushing-bass-133',
  'mlflow.source.name': '/home/codespace/.local/lib/python3.12/site-packages/ipykernel_launcher.py',
  'mlflow.source.type': 'LOCAL',
  'mlflow.user': 'codespace',
  'model': 'xgboost'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/1/d82b8c4499ca4f20a6bc26131d455856/artifacts', end_time=1730130185751, experiment_id='1', lifecycle_stage='active', run_id='d82b8c4499ca4f20a6bc26131d455856', run_name='blushing-bass-133', run_uuid='d82b8c4499ca4f20a6bc26131d455856', start_time=1730129852137, status='FAILED', user_id='codespace'>, inputs=<RunInputs: dataset_inputs=[]>>,
 <Run: data=<RunData: metrics={'best_iteration': 320.0,
  'rmse': 2.5783958849462065,
  'stopped_iteration': 370.0,
  'validation-rm

In [8]:
runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 10",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']}")

run id: d82b8c4499ca4f20a6bc26131d455856, rmse: 2.5783958849462065
run id: 730ce618f1d5499590a3008b364a0c14, rmse: 2.5783958849462065
run id: 908602715f834d978925e0c2450fd033, rmse: 2.5783987046175953
run id: eabbdaaef77143f4ba708f6844fb1c4b, rmse: 2.578402070804877
run id: 99c2c5fe36d04def99e8741c95855b9e, rmse: 2.578424661145273


promoting to model registry

In [17]:
import mlflow

top_run = None

for run in runs:
    # if top_run is None or run.data.metrics["rmse"] < top_run.data.metrics["rmse"]+.5:
    if run.info.run_id.startswith("7"):
        top_run = run

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.register_model(f"runs:/{run.info.run_id}/model", "duration-prediction")

Registered model 'duration-prediction' already exists. Creating a new version of this model...
2024/10/29 12:00:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: duration-prediction, version 3
Created version '3' of model 'duration-prediction'.


<ModelVersion: aliases=[], creation_timestamp=1730203231277, current_stage='None', description='', last_updated_timestamp=1730203231277, name='duration-prediction', run_id='99c2c5fe36d04def99e8741c95855b9e', run_link='', source='mlflow-artifacts:/1/99c2c5fe36d04def99e8741c95855b9e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

model transition


In [18]:
client.get_latest_versions("duration-prediction")

/tmp/ipykernel_9219/477942883.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions("duration-prediction")


[<ModelVersion: aliases=[], creation_timestamp=1730203231277, current_stage='None', description='', last_updated_timestamp=1730203231277, name='duration-prediction', run_id='99c2c5fe36d04def99e8741c95855b9e', run_link='', source='mlflow-artifacts:/1/99c2c5fe36d04def99e8741c95855b9e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>]

In [19]:
model_name = "duration-prediction"
latest_versions = client.get_latest_versions(model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/tmp/ipykernel_9219/3363948356.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(model_name)


In [20]:
client.transition_model_version_stage(
    name=model_name,
    version=version.version,
    stage="Production",
    archive_existing_versions=False,
)

/tmp/ipykernel_9219/306962077.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1730203231277, current_stage='Production', description='', last_updated_timestamp=1730203434091, name='duration-prediction', run_id='99c2c5fe36d04def99e8741c95855b9e', run_link='', source='mlflow-artifacts:/1/99c2c5fe36d04def99e8741c95855b9e/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>